In [0]:
dbutils.widgets.text("proc_date", "")
proc_date = dbutils.widgets.get("proc_date")

In [0]:
%run ../../../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
create_temp_view_with_clean_columns(
    spark,
    catalog_name,
    'udp_wcm_bronze_sap_bw',
    'sap_bw_wcm_prt_promo',
    proc_date,
    'temp_sap_prt_promo'
)

In [0]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {catalog_name}.udp_wcm_silver_sap_bw.sap_prt_promo
(
    hash_id BIGINT,
    rt_promo STRING,
    objvers STRING,
    changed STRING,
    rt_promoct STRING,
    rt_promoty STRING,
    rt_prslfrm DATE,
    rt_prslto DATE,
    rt_prcreap STRING,
    _bic_zbbuytype STRING,
    _bic_zbbprtven DOUBLE,
    _bic_zc_bbprof STRING,
    _bic_zc_bbtype STRING,
    _bic_zc_vpromo STRING,
    _bic_zc_bbstat STRING,
    loc_currcy STRING,
    _bic_zk_fm_km DOUBLE,
    _bic_zk_fm_th DOUBLE,
    _bic_zk_claim DOUBLE,
    _bic_zc_claimt STRING,
    proc_date DATE,
    start_date DATE,
    end_date DATE,
    file_creation_ts TIMESTAMP
    )
TBLPROPERTIES 
(
	DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE = TRUE,
	DELTA.AUTOOPTIMIZE.AUTOCOMPACT = TRUE
)
""")

In [0]:
spark.sql(f"""
CREATE OR REPLACE TEMP VIEW sap_prt_promo
AS
WITH temp AS (
SELECT * EXCEPT(rn) FROM (
        SELECT *, ROW_NUMBER() OVER (PARTITION BY rt_promo, DATE(proc_date) ORDER BY proc_date) rn FROM (
                SELECT IF(rt_promo = '', NULL, LTRIM('0',rt_promo)) AS rt_promo,
                        IF(objvers = '', NULL, objvers) AS objvers,
                        IF(changed = '', NULL, changed ) AS changed,
                        IF(rt_promoct = '', NULL, rt_promoct ) AS rt_promoct,
                        IF(rt_promoty = '', NULL, LTRIM('0',rt_promoty)) AS rt_promoty,
                        TO_DATE(IF(rt_prslfrm = '00000000', '20160101', rt_prslfrm), 'yyyyMMdd') AS rt_prslfrm,
                        TO_DATE(IF(rt_prslto = '00000000', '20160101', rt_prslto), 'yyyyMMdd') AS rt_prslto,
                        IF(rt_prcreap = '', NULL, rt_prcreap ) AS rt_prcreap,
                        IF(_bic_zbbuytype = '', NULL, _bic_zbbuytype ) AS _bic_zbbuytype,
                        COALESCE(NULLIF(CASE 
                        WHEN ABS(_bic_zbbprtven) < 1E-10 THEN '0'
                        WHEN _bic_zbbprtven = CAST(_bic_zbbprtven AS DECIMAL(38,0)) THEN CAST(CAST(_bic_zbbprtven AS DECIMAL(38,0)) AS STRING) 
                        ELSE CAST(_bic_zbbprtven AS STRING) 
                        END, 'None'), '') AS _bic_zbbprtven,
                        IF(_bic_zc_bbprof = '', NULL, _bic_zc_bbprof ) AS _bic_zc_bbprof,
                        IF(_bic_zc_bbtype = '', NULL, _bic_zc_bbtype ) AS _bic_zc_bbtype,
                        IF(_bic_zc_vpromo = '', NULL, _bic_zc_vpromo ) AS _bic_zc_vpromo,
                        IF(_bic_zc_bbstat = '', NULL, _bic_zc_bbstat ) AS _bic_zc_bbstat,
                        IF(loc_currcy = '', NULL, loc_currcy ) AS loc_currcy,
                        COALESCE(NULLIF(CASE 
                        WHEN ABS(_bic_zk_fm_km) < 1E-10 THEN '0'
                        WHEN _bic_zk_fm_km = CAST(_bic_zk_fm_km AS DECIMAL(38,18)) THEN CAST(CAST(_bic_zk_fm_km AS DECIMAL(38,18)) AS STRING) 
                        ELSE CAST(_bic_zk_fm_km AS STRING) 
                        END, 'None'), '') AS _bic_zk_fm_km,
                        COALESCE(NULLIF(CASE 
                        WHEN ABS(_bic_zk_fm_th) < 1E-10 THEN '0'
                        WHEN _bic_zk_fm_th = CAST(_bic_zk_fm_th AS DECIMAL(38,18)) THEN CAST(CAST(_bic_zk_fm_th AS DECIMAL(38,18)) AS STRING) 
                        ELSE CAST(_bic_zk_fm_th AS STRING) 
                        END, 'None'), '') AS _bic_zk_fm_th,
                        COALESCE(NULLIF(CASE 
                        WHEN ABS(_bic_zk_claim) < 1E-10 THEN '0'
                        WHEN _bic_zk_claim = CAST(_bic_zk_claim AS DECIMAL(38,18)) THEN CAST(CAST(_bic_zk_claim AS DECIMAL(38,18)) AS STRING) 
                        ELSE CAST(_bic_zk_claim AS STRING) 
                        END, 'None'), '') AS _bic_zk_claim,
                        IF(_bic_zc_claimt = '', NULL, _bic_zc_claimt ) AS _bic_zc_claimt,
                        proc_date AS proc_date,
                        file_creation_ts
                        FROM temp_sap_prt_promo
        ) a
) a WHERE rn = 1
),
main AS (
SELECT farm_fingerprint(CONCAT(
                    IFNULL(CAST(rt_promo AS STRING), ""), 
                    IFNULL(CAST(objvers AS STRING), ""), 
                    IFNULL(CAST(changed AS STRING), ""), 
                    IFNULL(CAST(rt_promoct AS STRING), ""),
                    IFNULL(CAST(rt_promoty AS STRING), ""),
                    IFNULL(CAST(rt_prslfrm AS STRING), ""),
                    IFNULL(CAST(rt_prslto AS STRING), ""),
                    IFNULL(CAST(rt_prcreap AS STRING), ""),
                    IFNULL(CAST(_bic_zbbuytype AS STRING), ""),
                    IFNULL(CASE WHEN CAST(_bic_zbbprtven AS DOUBLE) = CAST(CAST(_bic_zbbprtven AS DOUBLE) AS BIGINT) THEN CAST(CAST(_bic_zbbprtven AS BIGINT) AS STRING) ELSE TRIM(TRAILING '0' FROM(CAST(_bic_zbbprtven AS DECIMAL(38,10)))) END, ""),
                    IFNULL(CAST(_bic_zc_bbprof AS STRING), ""),
                    IFNULL(CAST(_bic_zc_bbtype AS STRING), ""),
                    IFNULL(CAST(_bic_zc_vpromo AS STRING), ""),
                    IFNULL(CAST(_bic_zc_bbstat AS STRING), ""),
                    IFNULL(CAST(loc_currcy AS STRING), ""),
                   IFNULL(CASE WHEN CAST(_bic_zk_fm_km AS DOUBLE) = CAST(CAST(_bic_zk_fm_km AS DOUBLE) AS BIGINT) THEN CAST(CAST(_bic_zk_fm_km AS BIGINT) AS STRING) ELSE TRIM(TRAILING '0' FROM(CAST(_bic_zk_fm_km AS DECIMAL(38,10)))) END, ""),
                   IFNULL(CASE WHEN CAST(_bic_zk_fm_th AS DOUBLE) = CAST(CAST(_bic_zk_fm_th AS DOUBLE) AS BIGINT) THEN CAST(CAST(_bic_zk_fm_th AS BIGINT) AS STRING) ELSE TRIM(TRAILING '0' FROM(CAST(_bic_zk_fm_th AS DECIMAL(38,10)))) END, "") ,
                    IFNULL(CASE WHEN CAST(_bic_zk_claim AS DOUBLE) = CAST(CAST(_bic_zk_claim AS DOUBLE) AS BIGINT) THEN CAST(CAST(_bic_zk_claim AS BIGINT) AS STRING) ELSE TRIM(TRAILING '0' FROM(CAST(_bic_zk_claim AS DECIMAL(38,10)))) END, "") ,
                    IFNULL(CAST(_bic_zc_claimt AS STRING), "")
		)) hash_id
        ,* 
        , DATE(proc_date) start_date FROM temp
),
main_2 AS (
SELECT * 
	, LAG(hash_id, 1) OVER (PARTITION BY rt_promo ORDER BY proc_date) hash_id_prev 
		FROM main
),
main_3 AS (
SELECT * EXCEPT(hash_id_prev) FROM main_2 WHERE (hash_id != hash_id_prev OR hash_id_prev IS NULL)
),
main_4 AS (
SELECT *, LEAD(DATE(proc_date)) OVER (PARTITION BY rt_promo ORDER BY proc_date ASC) end_date FROM main_3
)
SELECT * EXCEPT(end_date), IFNULL(date_add(end_date, -1), '2400-01-01') end_date FROM main_4
""")

In [0]:
spark.sql(f"""
MERGE INTO {catalog_name}.udp_wcm_silver_sap_bw.sap_prt_promo inf_tbl
USING (
    SELECT rt_promo, hash_id, start_date FROM sap_prt_promo
        WHERE end_date = DATE '2400-01-01' 
) tmp_tbl
ON inf_tbl.rt_promo = tmp_tbl.rt_promo AND inf_tbl.hash_id != tmp_tbl.hash_id AND inf_tbl.end_date = DATE '2400-01-01'
WHEN MATCHED THEN UPDATE SET inf_tbl.end_date = date_add(tmp_tbl.start_date, -1);
""")

In [0]:
spark.sql(f"""
MERGE INTO {catalog_name}.udp_wcm_silver_sap_bw.sap_prt_promo inf_tbl
	USING sap_prt_promo tmp_tbl
	ON inf_tbl.hash_id = tmp_tbl.hash_id  
	AND inf_tbl.end_date = DATE '2400-01-01'
	WHEN NOT MATCHED THEN INSERT *;
""")